#Conectamos a google drive e importamos librerias

In [ ]:
# Se conecta al Drive:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AX4XfWit2LhxQxYnF2wVYdYwDzzUd8h7fbw853leldR4NfimtsAj9HZfNlE


In [ ]:
from numpy.random import seed
seed(1)
import tensorflow
tensorflow.random.set_seed(2)
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import pickle #para guardado de datos

from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import Sequential
from keras.layers.core import Dense, Flatten, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D,Conv2D, AveragePooling2D, BatchNormalization, Dropout, MaxPooling2D,RandomFlip,RandomRotation

import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

# Para importar el Helper
%cd "/content/drive/Shareddrives/Redes neuronales/TP3"
!pip install import-ipynb
import import_ipynb
from Helper import *

/content/drive/Shareddrives/Redes neuronales/TP3
  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-py3-none-any.whl size=2975 sha256=4ec66d06dc486241e40897a897f40d0d6c28b56a0d54e4da33d0fe9b5812ae0c
  Stored in directory: /root/.cache/pip/wheels/b1/5e/dc/79780689896a056199b0b9f24471e3ee184fbd816df355d5f0
Successfully built import-ipynb
importing Jupyter notebook from Helper.ipynb
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 16)   448         ['input_1[0][0]']                
                                                                                                

# Cargamos los datos

In [ ]:
X_train = np.load("/content/drive/Shareddrives/Redes neuronales/TP3/Dataset original/X_train.npy")
Y_train = np.load("/content/drive/Shareddrives/Redes neuronales/TP3/Dataset original/y_train.npy")
X_test = np.load("/content/drive/Shareddrives/Redes neuronales/TP3/Dataset original/X_test.npy")

# Dividimos entre datasets de entrenamiento y validación

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(X_train, Y_train, test_size = 0.1)

Con la función **train_test_split** se divide el X_train en el conjunto de validación y de entrenamiento.

# Paso a float

In [ ]:
x_train = x_train.astype('float32')
x_valid = x_valid.astype('float32')
x_test = X_test.astype('float32')
x_train /= 255
x_valid /= 255
x_test /= 255

Se normalizan los datos.

In [ ]:
y_train_categorical = to_categorical(y_train,100)
y_val_categorical = to_categorical(y_valid,100)

**to_categorical** convierte un vector de clase (enteros) en una matriz de clase binaria.

# Funciones

In [ ]:
def CNN_MobileNet(shape, cant_Neuronas, loss_function, optimizer_chosen, principal_metrics, set_weights = "imagenet", layer_trainable = True):
  # Se instancia la arquitectura de MobileNET
  model_resnet = tf.keras.applications.MobileNet(include_top = False, weights= set_weights, input_shape = shape, pooling = 'avg', dropout=0.001, classes=100)

  if layer_trainable == False:
    # Si layer_trainable es False, quiere decir que al entrenar el modelo, los pesos no cambian.
    for layer in model_resnet.layers:
      layer.trainable = False
    print('Cero pesos entrenables')

  print("Capas convolucionales del modelo:")
  # Se printean las capas convolucionales del modelo
  model_resnet.summary()

  # Se crean e inicializan los Layers del modelo
  input_layer = model_resnet.layers[0].input
  output_resnet = model_resnet.layers[-1].output

  x = GlobalAveragePooling2D()(output_resnet)
  x = Dense(cant_Neuronas, activation = "softmax")(x)

  # Se crea el modelo completo
  model = Model(inputs = input_layer, outputs = x)
  print('Modelo completo:')
  model.summary()

  # Se compila el modelo
  model.compile(loss = loss_function, optimizer = optimizer_chosen, metrics = principal_metrics)

  return model

**CNN_MobileNET:** Se usa el modelo dado en clase con la función de MobileNET.

- include_top --> Booleano, si se debe incluir la capa completamente conectada en la parte superior de la red.
- weights --> Se cargan los pesos pre-entrenados de la librería de imagenet.
- input_shape --> Hay que especificarlo porque include_top es False
- pooling --> Modo pooling opcional
- dropout --> Ratio Dropout
- classes --> son 100 clases

#Entrenamos el modelo

- tf.keras.layers.RandomCrop
- tf.keras.layers.RandomFlip
- tf.keras.layers.RandomTranslation
- tf.keras.layers.RandomRotation
- tf.keras.layers.RandomZoom
- tf.keras.layers.RandomHeight
- tf.keras.layers.RandomWidth
- tf.keras.layers.RandomContrast

In [ ]:
def data_augmentation(x_train_data,y_train_data,random_flip=True,mode_flip ="horizontal_and_vertical", random_rotation=True,factor_rotation = 0.2,random_crop=False,height_crop=10,width_crop=10,random_translation=False,height_translation=10,width_translation=10,random_zoom=False,height_zoom=10,width_zoom=10,random_contrast=False,factor_contrast=0.2):
  data_augmentation = tf.keras.Sequential()
  if random_flip: data_augmentation.add(layers.RandomFlip(mode_flip))
  if random_rotation: data_augmentation.add(layers.RandomRotation(factor_rotation))    
  if random_crop: layers.RandomCrop(height_crop,width_crop) #crops image
  if random_translation: layers.RandomTranslation(height_translation, width_translation) 
  if random_zoom: layers.RandomZoom(height_zoom, width_zoom)
  if random_contrast: layers.RandomContrast(factor_contrast)

  x_train_augmented = np.append(x_train_data,data_augmentation(x_train),axis=0)
  y_train_augmented = np.append(y_train_categorical,y_train_categorical,axis=0)

  return x_train_augmented,y_train_augmented

Se aplica Data Augmentation

In [ ]:
x_train_augmented,y_train_augmented = data_augmentation(x_train,y_train_categorical)

In [ ]:
# Parametros de creación de modelo:
shape = (x_train_augmented.shape[1], x_train_augmented.shape[2], x_train_augmented.shape[3])
setten_weights = "imagenet"
cant_Neuronas = 100
layer_trainable = True
# Parametros para la compilación del modelo (Se hace todo en la misma funcion):
loss_func = 'categorical_crossentropy'
optimizer_chosen = "Adam"
principal_metrics = ["accuracy"]

In [ ]:
batch_size = 500
epocs = 30
verbose = 1
monitor='val_accuracy'
filepath = f'/content/drive/Shareddrives/Redes neuronales/TP3/Modelos entrenados/modelCNN_MobileNET_batch_size_{batch_size}_epocs{epocs}_l_trainable_{layer_trainable}_dataAUg_FALSE'

Se llamada a la función creadora de MobileNet

In [ ]:
model = CNN_MobileNet(shape, cant_Neuronas, loss_func, optimizer_chosen, principal_metrics, setten_weights, layer_trainable)

Capas convolucionales del modelo:
Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv1 (Conv2D)              (None, 16, 16, 32)        864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 16, 16, 32)       128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 16, 16, 32)        0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 16, 16, 32)       288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 16, 16, 32)       128       
 ation)       

Se entrena el modelo con la función **train** del Helper

In [ ]:
history, model = train(model, monitor, filepath, epocs, batch_size, x_train,y_train_categorical, x_valid, y_val_categorical, verbose, loss_func, optimizer_chosen, principal_metrics)

Epoch 1/30
90/90 [==============================] - ETA: 0s - loss: 3.3884 - accuracy: 0.2354
Epoch 00001: val_loss improved from inf to 6.24715, saving model to /content/drive/Shareddrives/Redes neuronales/TP3/Modelos entrenados/modelCNN_MobileNET_batch_size_500_epocs30_l_trainable_True_dataAUg_FALSE
INFO:tensorflow:Assets written to: /content/drive/Shareddrives/Redes neuronales/TP3/Modelos entrenados/modelCNN_MobileNET_batch_size_500_epocs30_l_trainable_True_dataAUg_FALSE/assets
90/90 [==============================] - 61s 616ms/step - loss: 3.3884 - accuracy: 0.2354 - val_loss: 6.2472 - val_accuracy: 0.0646
Epoch 2/30
90/90 [==============================] - ETA: 0s - loss: 2.1502 - accuracy: 0.4324
Epoch 00002: val_loss improved from 6.24715 to 3.82870, saving model to /content/drive/Shareddrives/Redes neuronales/TP3/Modelos entrenados/modelCNN_MobileNET_batch_size_500_epocs30_l_trainable_True_dataAUg_FALSE
INFO:tensorflow:Assets written to: /content/drive/Shareddrives/Redes neuron